<img width="8%" alt="OWID.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/OWID.png" style="border-radius: 15%">

# OWID - Visualize Human Development Index
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=OWID+-+Visualize+Human+Development+Index:+Error+short+description">Bug report</a>

**Tags:** #owid #dash #dashboard #plotly #naas #asset #analytics #dropdown #callback #bootstrap #snippet

**Author:** [Zihui Ouyang](https://www.linkedin.com/in/zihui-ouyang-539626227/)

**Last update:** 2023-08-07 (Created: 2023-08-07)

**Description:** This notebook creates an interactive plot using Dash app infrastructure with OWID's HDI data.

**References:**
- https://github.com/owid/owid-datasets/tree/master/datasets/Human%20Development%20Index%20-%20UNDP
- https://stackoverflow.com/questions/70886359/dash-python-making-subplots-when-multiple-parameters-are-selected
- https://dash-example-index.herokuapp.com/line-charts

## Input

### Import libraries

In [ ]:
try:
    import dash
    import os
except:
    !pip install dash --user
    import dash
try:
    import dash_bootstrap_components as dbc
except:
    !pip install dash_bootstrap_components --user
    import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px

### Setup variables
- `DASH_PORT`: specify a port number for Dash
- `url`: URL to get data from Excel
- `title`: App title

In [ ]:
DASH_PORT = 8050
url = "https://raw.githubusercontent.com/owid/owid-datasets/master/datasets/Human%20Development%20Index%20-%20UNDP/Human%20Development%20Index%20-%20UNDP.csv"
title = "Human Development Index"

## Model

### Initialize Dash app
The `os.environ.get("JUPYTERHUB_USER")` is used to access the environment variable `JUPYTERHUB_USER` already stored into your Naas Lab.

In [ ]:
app = dash.Dash(
    title=title,
    requests_pathname_prefix=f'/user/{os.environ.get("JUPYTERHUB_USER")}/proxy/{DASH_PORT}/',
    external_stylesheets=[dbc.themes.BOOTSTRAP],
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1.0"}
    ],
)

#app = dash.Dash(title = "Human Development Index") if you are not in Naas

### Get Data

In [ ]:
contents_df = pd.read_csv(url, header = 0) 

### Create Dash app

In [ ]:
app.layout = html.Div(
    [
        html.H4("Human Development Index up to 2017"),
        html.P("Select country"),
        dcc.Dropdown(
            id="country",
            options= contents_df.Entity.unique(),
            value="United States"
        ),
        dcc.RangeSlider(id='slider', min=1980, max=2017, value=[1980, 2017],
               marks={x: str(x) for x in [1980, 1987, 1994, 2001, 2008, 2017]}),
        dcc.Graph(id="Human Development Index (UNDP)", figure={}, style={'display': 'none'})
    ]
)

@callback(
    Output("Human Development Index (UNDP)", 'figure'),
    Output("Human Development Index (UNDP)", 'style'),
    Input('country', 'value'),
    Input('slider', 'value')
)

def update_graph(country, year):
    country_list = [country]
    dff = (contents_df["Entity"].isin(country_list)) & (contents_df["Year"] <= year[1]) & (contents_df["Year"] >= year[0])
        
    figures = px.line(contents_df[dff], x='Year', y="Human Development Index (UNDP)", markers = True).update_layout(
                                plot_bgcolor='rgba(0, 0, 0, 0)', height= 600)
    styles = {'display': 'block'}   
    return figures, styles

## Output

### Generate URL and show logs

In [ ]:
if __name__ == "__main__":
    app.run_server(proxy=f"http://127.0.0.1:{DASH_PORT}::https://app.naas.ai")